In [52]:
!pip install mistralai evaluate python-dotenv openai together pydantic anthropic tqdm sentence_transformers bert_score rouge Levenshtein

In [32]:
reviews = [
    # Thermal Mugs
    "Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.",
    "The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.",
    "Love the sleek design of my Zojirushi mug, and it fits perfectly in my car cup holder!",
    "It’s lightweight but keeps my drinks at the right temperature for hours with Hydro Flask.",
    "No more cold coffee! This Yeti thermal mug does the job.",
    "It’s easy to clean, and the thermal insulation works like a charm.",
    "The handle makes it easy to carry, and it doesn’t spill.",
    "Not great for keeping drinks cold, but excellent for hot beverages.",
    "I wish it were bigger, but it’s perfect for my morning tea.",
    "I accidentally dropped it, and it didn’t dent! Very sturdy.",
    "The rubber seal around the lid came loose after a few weeks. Disappointing.",
    "Great for both coffee and soup—keeps them warm for hours.",
    "The exterior stays cool, even when my drink is piping hot inside.",
    "I love the color options, and it’s great for on-the-go.",
    "Keeps ice water cold for hours, even in hot weather!",
    "It’s a little tricky to open one-handed, but overall, a great mug.",
    "The size is perfect for travel, and it keeps drinks hot all day.",
    "It doesn’t leak, even when I toss it in my bag. Highly recommend Contigo.",
    "The lid is a little tight, but the mug works well for keeping drinks warm.",
    "Very stylish and functional! I get compliments all the time.",
    "Keeps my coffee scalding hot for longer than any mug I’ve owned with Zojirushi.",
    "Great value for the price. Works just as well as more expensive brands.",
    "The mug is lightweight and easy to carry around.",
    "It fits perfectly under my single-serve coffee machine!",
    "Durable, sleek, and it does exactly what it’s supposed to.",

    # Dishwasher Detergents
    "My dishes come out sparkling clean every time with Cascade. Love this detergent!",
    "It works well on glass, but I’ve noticed spots on my silverware.",
    "Great for tough, greasy messes. Leaves no residue! Thanks, Finish.",
    "This detergent smells amazing and leaves my dishwasher fresh.",
    "It’s a little pricey, but my dishes have never looked better with Cascade Platinum.",
    "Gets rid of even the most stubborn baked-on food. Highly recommend Finish Quantum.",
    "Not the best on hard water stains, but otherwise it works great.",
    "My dishes have never been so spotless after a wash!",
    "It’s very effective, but I wish it came in a fragrance-free version.",
    "Cuts through grease like a dream. No more pre-rinsing with Cascade Complete.",
    "This detergent doesn’t leave any residue on plastic, which I love.",
    "My glasses come out clear and sparkling every single time.",
    "It doesn’t work well with my eco dishwasher. Dishes aren’t as clean.",
    "Very efficient—gets rid of food stains and smells with no problem.",
    "I noticed some streaks on my glassware, but overall it works well.",
    "Leaves my dishes spotless and my machine smelling fresh.",
    "A great, eco-friendly option that actually works!",
    "It’s a little hard on some of my delicate dishware.",
    "This is the only detergent that works on my hard water stains.",
    "No need to rewash dishes after using this—so efficient!",
    "Perfect for everyday use. My dishes are clean and shiny.",
    "Leaves a chemical smell, but it’s effective at cleaning.",
    "A bit expensive, but worth it for the spotless results.",
    "No more streaks or water spots! Best dishwasher detergent ever.",
    "My silverware and dishes look brand new after every wash.",

    # Sunscreens
    "Absorbs quickly and doesn’t leave a greasy residue. Great for daily use with Neutrogena.",
    "The scent is a little strong, but it protects well with Coppertone.",
    "Perfect for sensitive skin! No breakouts or irritation with La Roche-Posay.",
    "A bit thick to apply, but once it’s on, it stays all day.",
    "I love the lightweight formula of CeraVe, perfect for wearing under makeup.",
    "Doesn’t leave a white cast, even on darker skin tones.",
    "The spray bottle makes it super easy to apply on the go.",
    "This sunscreen saved me from burning on a beach vacation with Banana Boat!",
    "It’s waterproof, which is a must for pool days. Highly recommend Hawaiian Tropic.",
    "A bit pricey, but the protection it provides is worth every penny with Supergoop.",
    "This is my go-to sunscreen for both my face and body with Neutrogena.",
    "It’s a little greasy, but it gets the job done in strong sun.",
    "No weird scent, and it goes on smooth. Love this product!",
    "Perfect for outdoor activities—no sunburn, even after hours outside.",
    "It’s great for kids! No irritation, and it’s easy to apply with Blue Lizard.",
    "A little too heavy for my face, but works perfectly for the body.",
    "The texture is nice and light, not sticky at all.",
    "This sunscreen doesn’t clog my pores, which is a huge plus with EltaMD.",
    "I’ve tried a lot of sunscreens, and this one offers the best protection with La Roche-Posay.",
    "It leaves a slight sheen, but I love how protected my skin feels.",
    "This formula doesn’t dry out my skin like others do.",
    "It’s great under makeup—no pilling or greasiness.",
    "I wish it were more affordable, but it’s worth it for the protection.",
    "Very effective, even after swimming for hours.",
    "My skin stays soft and protected all day with Neutrogena sunscreen."

     # Powder Detergents for Laundry
    "Gets my clothes fresh and clean every time. No lingering odor with Tide.",
    "It dissolves well, even in cold water. My whites have never been brighter thanks to Ariel.",
    "A little pricey, but worth it for the excellent stain removal power of Persil.",
    "This powder leaves a residue on darker clothes. Not a fan of OMO.",
    "Great for sensitive skin! No itching or redness after using Seventh Generation.",
    "I love how eco-friendly this detergent is. It’s a big plus for me with Ecover.",
    "I don’t need fabric softener anymore—this leaves my clothes so soft!",
    "My laundry has never smelled so fresh, and it lasts for days with Gain.",
    "It’s not the best for heavy stains but works great for daily washes.",
    "Great value for the price. This box lasts forever! Thanks, Arm & Hammer.",
    "Perfect for my workout gear—gets rid of all the sweat smells.",
    "Leaves a bit of powder behind in the machine, but it cleans well.",
    "I’ve been using it for years, and Tide never disappoints.",
    "Not as effective in hard water areas, but still decent.",
    "My go-to detergent for all of my family’s laundry needs.",
    "I noticed some fading in my darker clothes after a few washes.",
    "It’s gentle on my baby’s clothes and skin with Dreft.",
    "Very effective at removing mud and grass stains from the kids’ clothes.",
    "I like the scent, but it might be too strong for some.",
    "No complaints so far! My clothes feel clean and fresh.",
    "Works just as well as liquid detergents but at a lower cost.",
    "A bit too perfumed for my taste, but it gets the job done.",
    "My clothes are noticeably softer and smell better than before.",
    "The box is hard to pour from, but the detergent works well.",
    "This is my new favorite detergent. So much better than the leading brand!",
]

product_offers = [
    # Thermal Mugs
    "Contigo Workday Travel Mug – Keeps Coffee Hot for Hours!",
    "Zojirushi Sleek Travel Mug – Perfect Fit for Car Holders",
    "Hydro Flask Lightweight Insulation Mug – Stay Warm for Hours",
    "Yeti Thermal Mug – No More Cold Coffee!",
    "Contigo All-Day Heat Retention Mug – Ideal for Travel",
    "Contigo Leak-Proof Mug – Toss in Your Bag with Confidence",
    "Zojirushi Scalding Hot Coffee Mug – Best Insulation Yet",

    # Dishwasher Detergents
    "Cascade Sparkling Clean Detergent – Your Dishes Will Shine",
    "Finish Detergent – Tough on Grease, No Residue Left",
    "Cascade Platinum Detergent – Pricey, But Worth It for Results",
    "Finish Quantum Detergent – Stubborn Food Stains Gone",
    "Cascade Complete Detergent – No Pre-Rinsing Needed for Grease",

    # Sunscreens
    "Neutrogena Daily Sunscreen – Quick Absorption, No Grease",
    "Coppertone Suncream – Strong Scent, Strong Protection",
    "La Roche-Posay Sensitive Skin Sunscreen – No Breakouts",
    "CeraVe Lightweight Sunscreen – Perfect Under Makeup",
    "Banana Boat Beach-Saver Sunscreen – No Burns, Just Fun",
    "Hawaiian Tropic Waterproof Sunscreen – Pool Day Essential",
    "Supergoop Premium Sunscreen – Worth Every Penny",
    "Neutrogena Face & Body Sunscreen – All-Purpose Protection",
    "EltaMD Pore-Friendly Sunscreen – Protection Without Clogging",
    "La Roche-Posay Suncream – The Best in Sun Protection",
    "Neutrogena Sunscreen – Soft, Protected Skin All Day",

    # Powder Detergents for Laundry
    "Tide Powder Detergent – Fresh, Clean Clothes Every Time",
    "Ariel Powder Detergent – Whites Brighter, Even in Cold Water",
    "Persil Powder Detergent – Powerful Stain Removal",
    "OMO Powder Detergent – Leaves Residue on Dark Clothes",
    "Seventh Generation Powder Detergent – Great for Sensitive Skin",
    "Ecover Eco-Friendly Detergent – Perfect for the Eco-Conscious",
    "Gain Powder Detergent – Fresh-Smelling Laundry for Days",
    "Arm & Hammer Powder Detergent – Great Value, Lasts Forever",
    "Dreft Baby Powder Detergent – Gentle on Baby Clothes"
]

# Creating product definition and model running functions

In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field
from typing import List, Literal


class Product(BaseModel):
    product_category: Literal[
        "Powder Detergents for Laundry",
        "Thermal Mugs",
        "Dishwasher Detergents",
        "Sunscreens",
        "Nappies",
        "Others"
    ] = Field(
        description="The category of the product.",
        alias="product category"
    )
    # product_category: str = Field(
    #     description="The category of the product.",
    #     alias="product category"
    # )
    brand: str = Field(
        description="The brand of the product, or 'N/A' if not applicable.",
        alias="brand"
    )
    
    other_keywords: List[str] = Field(
        description="A list of other keywords associated with the product.",
        alias="other keywords"
    )

In [43]:
from mistralai import Mistral
from openai import OpenAI
from dotenv import load_dotenv
import os
from together import Together
from anthropic import Anthropic

load_dotenv()


together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client_mistral = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))
client_anthropic = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

conversation_template = [
    {
        "role": "system", 
        "content": "Given the user review of the product, extract the following information: product category, brand, and other keywords, which are associated with the product."
    }, 
    {
        "role": "system", 
        "content": f"""
            Answer using only JSON format with the following JSON schema:

            {Product.model_json_schema()}
        """.lstrip()
    },
]

def run_mistral(user_message, model="mistral-large-latest"):
    messages = conversation_template[:1] + [{"role": "user", "content": user_message}] + conversation_template[1:] 
    #messages = [{"role": "user", "content": user_message}]
    chat_response = client_mistral.chat.complete(
        model=model,
        messages=messages,
        response_format={
            "type": "json_object",
            "json_schema": {"name": "product", "schema": Product.model_json_schema(),}
        },
    )
    return chat_response.choices[0].message.content


def run_openai(user_message, model="gpt-4o"):
    messages = conversation_template[:1] + [{"role": "user", "content": user_message}] + conversation_template[1:]
    chat_response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={
            "type": "json_object",
            # "json_schema": {"name": "product", "schema": Product.model_json_schema(),}
        }
    )
    return chat_response.choices[0].message.content

def run_together(user_message, model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"):
    messages = conversation_template[:1] + [{"role": "user", "content": user_message}] + conversation_template[1:]
    chat_response = together_client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={
            "type": "json_object",
            "json_schema": {"name": "product", "schema": Product.model_json_schema(),}
        },
    )
    return chat_response.choices[0].message.content

def run_anthropic(user_message, model="claude-3-5-sonnet-20241022"):
    messages = [{"role": "user", "content": user_message}]
    chat_response = client_anthropic.messages.create(
        model=model,
        messages=messages,
        system="\n".join([m["content"] for m in conversation_template]),
        max_tokens=1024
        # response_format={
        #     "type": "json_object",
        #     "schema": Product.model_json_schema(),
        # },
    )
    
    return chat_response.content[0].text

# Extracting product information from reviews

In [44]:
from tqdm import tqdm
import json

responses = []
def write_response_for_review(
    review: str
):
    print(f"Review: {review}")
    try:
        response_mistral = json.loads(run_mistral(review))
    except Exception as e:
        print(f"Mistral error: {e}")
        response_mistral = None
    
    try:
        response_openai = json.loads(run_openai(review))
    except Exception as e:
        print(f"OpenAI error: {e}")
        response_openai = None
        
    try:
        response_together = json.loads(run_together(review))
    except Exception as e:
        print(f"Together error: {e}")
        response_together = None
        
    try:
        response_anthropic = json.loads(run_anthropic(review))
    except Exception as e:
        print(f"Anthropic error: {e}")
        response_anthropic = None
    
    responses.append({
        "review": review,
        "mistral": response_mistral,
        "openai": response_openai,
        "together": response_together,
        "anthropic": response_anthropic,
    })

def write_response_for_product(
    product: str
):
    print(f"Product: {product}")
    try:
        response_mistral = json.loads(run_mistral(product))
    except Exception as e:
        print(f"Mistral error: {e}")
        response_mistral = None
    
    try:
        response_openai = json.loads(run_openai(product))
    except Exception as e:
        print(f"OpenAI error: {e}")
        response_openai = None
        
    try:
        response_together = json.loads(run_together(product))
    except Exception as e:
        print(f"Together error: {e}")
        response_together = None
        
    try:
        response_anthropic = json.loads(run_anthropic(product))
    except Exception as e:
        print(f"Anthropic error: {e}")
        response_anthropic = None
    
    responses.append({
        "product": product,
        "mistral": response_mistral,
        "openai": response_openai,
        "together": response_together,
        "anthropic": response_anthropic,
    })

# for each test case
tqdm._instances.clear()
for review in tqdm(reviews):
    write_response_for_review(review)
    
for product in tqdm(product_offers):
    write_response_for_product(product)

with open('js_zl_nk_js_responses.json', 'w') as f:
    json.dump(responses, f, indent=4)


  0%|          | 0/99 [00:00<?, ?it/s]

Review: Keeps my coffee hot for hours—just what I need for long workdays. Thanks, Contigo.


  1%|          | 1/99 [00:06<11:14,  6.88s/it]

Review: The lid isn’t leak-proof, but it keeps drinks warm for a decent amount of time.


  2%|▏         | 2/99 [00:14<11:47,  7.30s/it]

...

 94%|█████████▍| 30/32 [04:28<00:17,  8.64s/it]

Product: Arm & Hammer Powder Detergent – Great Value, Lasts Forever


 97%|█████████▋| 31/32 [04:42<00:10, 10.16s/it]

Product: Dreft Baby Powder Detergent – Gentle on Baby Clothes


100%|██████████| 32/32 [04:50<00:00,  9.08s/it]


In [ ]:
# Load responses from JSON file
with open('js_zl_nk_js_responses.json', 'r') as f:
    responses = json.load(f)

# Comparing similarity of responses from different models

In [66]:
import pandas as pd


# prompt: write code for comparing one review from results with one product from products and add in these comparison extracted keywords with lower weight additionally to categories and brand comparison

def compare_json_objects_with_keywords(obj1, obj2, keys_to_compare={'other keywords', 'brand', 'product category'}):
    if obj1 is None or obj2 is None:
        return 0
    
    identical_fields = 0
    common_keys = set(obj1.keys()) & set(obj2.keys() & keys_to_compare)
    for key in common_keys:
        if key == 'other keywords':
            if isinstance(obj1.get(key), list) and isinstance(obj2.get(key), list):
                common_keywords = set(obj1.get(key)) & set(obj2.get(key))
                identical_fields += len(common_keywords) * 0.3  # Keywords have lower weight
            else:
              identical_fields += 0
        else:
          identical_fields += obj1[key] == obj2[key]
    #percentage_identical = (identical_fields / max(len(obj1.keys()), 1)) * 100
    percentage_identical = (identical_fields / max(len(keys_to_compare), 1)) * 100

    return percentage_identical

# for each review and product find similarity between different model responses
similarity = pd.DataFrame(columns=['type', 'mistral-openai', 'mistral-together', 'mistral-anthropic', 'openai-together', 'openai-anthropic', 'together-anthropic'])
for response in responses:
    type = 'review' if 'review' in response else 'product'
    mistral_openai = compare_json_objects_with_keywords(response['mistral'], response['openai'])
    mistral_together = compare_json_objects_with_keywords(response['mistral'], response['together'])
    mistral_anthropic = compare_json_objects_with_keywords(response['mistral'], response['anthropic'])
    openai_together = compare_json_objects_with_keywords(response['openai'], response['together'])
    openai_anthropic = compare_json_objects_with_keywords(response['openai'], response['anthropic'])
    together_anthropic = compare_json_objects_with_keywords(response['together'], response['anthropic'])
    
    similarity = pd.concat(
        [
            similarity, 
            pd.DataFrame(
                [[type, mistral_openai, mistral_together, mistral_anthropic, openai_together, openai_anthropic, together_anthropic]],
                columns=['type', 'mistral-openai', 'mistral-together', 'mistral-anthropic', 'openai-together', 'openai-anthropic', 'together-anthropic']
            )
        ],
        ignore_index=True
    )
    
similarity.to_csv('js_zl_nk_js_similarity_1.csv', index=False)

print("Mean similarity between Mistral and OpenAI:", similarity['mistral-openai'].mean())
print("Mean similarity between Mistral and Together:", similarity['mistral-together'].mean())
print("Mean similarity between Mistral and Anthropic:", similarity['mistral-anthropic'].mean())
print("Mean similarity between OpenAI and Together:", similarity['openai-together'].mean())
print("Mean similarity between OpenAI and Anthropic:", similarity['openai-anthropic'].mean())
print("Mean similarity between Together and Anthropic:", similarity['together-anthropic'].mean())
similarity

Mean similarity between Mistral and OpenAI: 80.07633587786259
Mean similarity between Mistral and Together: 68.04071246819339
Mean similarity between Mistral and Anthropic: 72.44274809160305
Mean similarity between OpenAI and Together: 70.20356234096693
Mean similarity between OpenAI and Anthropic: 71.80661577608141
Mean similarity between Together and Anthropic: 63.94402035623409


,type,mistral-openai,mistral-together,mistral-anthropic,openai-together,openai-anthropic,together-anthropic
0,review,96.666667,86.666667,86.666667,86.666667,86.666667,76.666667
1,review,76.666667,96.666667,66.666667,76.666667,66.666667,76.666667
2,review,86.666667,86.666667,86.666667,86.666667,86.666667,86.666667
3,review,86.666667,76.666667,76.666667,76.666667,76.666667,76.666667
4,review,76.666667,66.666667,76.666667,76.666667,86.666667,76.666667
...,...,...,...,...,...,...,...
126,product,76.666667,66.666667,66.666667,66.666667,66.666667,66.666667
127,product,86.666667,63.333333,66.666667,53.333333,66.666667,33.333333
128,product,66.666667,86.666667,66.666667,66.666667,76.666667,66.666667
129,product,86.666667,86.666667,66.666667,86.666667,66.666667,66.666667


In [69]:
# prompt: add comparison when if product has different cateogry there is no similarity, but if it is the same it is about 30%, if has the same category and brand, similarity is about 75%

def compare_json_objects_with_keywords(obj1, obj2, keys_to_compare={'other keywords', 'brand', 'product category'}):
    if obj1 is None or obj2 is None:
        return 0
    
    if obj1.get('product category') != obj2.get('product category'):
        return 0  # No similarity if categories don't match

    similarity_score = 0
    common_keys = set(obj1.keys()) & set(obj2.keys() & keys_to_compare)

    if obj1.get('product category') == obj2.get('product category'):
        similarity_score += 0.3  # Base similarity for matching categories

    if obj1.get('brand') == obj2.get('brand'):
        similarity_score += 0.4  # Additional similarity for matching brands

    for key in common_keys:
        if key == 'other keywords':
            if isinstance(obj1.get(key), list) and isinstance(obj2.get(key), list):
                common_keywords = set(obj1.get(key)) & set(obj2.get(key))
                similarity_score += len(common_keywords) * 0.05  # Keywords have lower weight
            else:
                similarity_score += 0


    return min(100, round(similarity_score * 100))


# for each review and product find similarity between different model responses
similarity = pd.DataFrame(columns=['type', 'mistral-openai', 'mistral-together', 'mistral-anthropic', 'openai-together', 'openai-anthropic', 'together-anthropic'])
for response in responses:
    type = 'review' if 'review' in response else 'product'
    mistral_openai = compare_json_objects_with_keywords(response['mistral'], response['openai'])
    mistral_together = compare_json_objects_with_keywords(response['mistral'], response['together'])
    mistral_anthropic = compare_json_objects_with_keywords(response['mistral'], response['anthropic'])
    openai_together = compare_json_objects_with_keywords(response['openai'], response['together'])
    openai_anthropic = compare_json_objects_with_keywords(response['openai'], response['anthropic'])
    together_anthropic = compare_json_objects_with_keywords(response['together'], response['anthropic'])

    similarity = pd.concat(
        [
            similarity,
            pd.DataFrame(
                [[type, mistral_openai, mistral_together, mistral_anthropic, openai_together, openai_anthropic, together_anthropic]],
                columns=['type', 'mistral-openai', 'mistral-together', 'mistral-anthropic', 'openai-together', 'openai-anthropic', 'together-anthropic']
            )
        ],
        ignore_index=True
    )

similarity.to_csv('js_zl_nk_js_similarity_2.csv', index=False)

print("Mean similarity between Mistral and OpenAI:", similarity['mistral-openai'].mean())
print("Mean similarity between Mistral and Together:", similarity['mistral-together'].mean())
print("Mean similarity between Mistral and Anthropic:", similarity['mistral-anthropic'].mean())
print("Mean similarity between OpenAI and Together:", similarity['openai-together'].mean())
print("Mean similarity between OpenAI and Anthropic:", similarity['openai-anthropic'].mean())
print("Mean similarity between Together and Anthropic:", similarity['together-anthropic'].mean())
similarity

Mean similarity between Mistral and OpenAI: 68.62595419847328
Mean similarity between Mistral and Together: 54.541984732824424
Mean similarity between Mistral and Anthropic: 68.16793893129771
Mean similarity between OpenAI and Together: 58.587786259541986
Mean similarity between OpenAI and Anthropic: 64.50381679389314
Mean similarity between Together and Anthropic: 52.86259541984733


,type,mistral-openai,mistral-together,mistral-anthropic,openai-together,openai-anthropic,together-anthropic
0,review,85,80,80,80,80,75
1,review,75,85,70,75,70,75
2,review,80,80,80,80,80,80
3,review,80,75,75,75,75,75
4,review,75,70,75,75,80,75
...,...,...,...,...,...,...,...
126,product,75,70,70,70,70,70
127,product,80,0,70,0,70,0
128,product,70,80,70,70,75,70
129,product,80,80,70,80,70,70


# Merging results from OpenAI, Mistral, Together, and Anthropic models for reviews and products

In [76]:
import json

leading_model = 'openai'  # Choose the model with the highest similarity score
merged_products = []

for response in responses:
    if 'product' not in response:
        continue
    
    product = response['product']
    model_keywords = {
        *(response.get('mistral', {}) or {}).get('other keywords', []),
        *(response.get('openai', {}) or {}).get('other keywords', []),
        *(response.get('together', {}) or {}).get('other keywords', []),
        *(response.get('anthropic', {}) or {}).get('other keywords', [])
    }
    
    merged_product = {
        "product": product,
        "product category": response[leading_model].get("product category", "N/A"),
        "brand": response[leading_model].get("brand", "N/A"),
        "other keywords": list(model_keywords)
    }

    merged_products.append(merged_product)

merged_reviews = []
for response in responses:
    if 'review' not in response:
        continue
    
    review = response['review']
    model_keywords = {
        *(response.get('mistral', {}) or {}).get('other keywords', []),
        *(response.get('openai', {}) or {}).get('other keywords', []),
        *(response.get('together', {}) or {}).get('other keywords', []),
        *(response.get('anthropic', {}) or {}).get('other keywords', [])
    }
    
    merged_review = {
        "review": review,
        "product category": response[leading_model].get("product category", "N/A"),
        "brand": response[leading_model].get("brand", "N/A"),
        "other keywords": list(model_keywords)
    }

    merged_reviews.append(merged_review)
    
with open('js_zl_nk_js_merged_products.json', 'w') as f:
    json.dump(merged_products, f, indent=4)

with open('js_zl_nk_js_merged_reviews.json', 'w') as f:
    json.dump(merged_reviews, f, indent=4)

In [ ]:
with open('js_zl_nk_js_merged_products.json', 'r') as f:
    merged_products = json.load(f)
    
with open('js_zl_nk_js_merged_reviews.json', 'r') as f:
    merged_reviews = json.load(f)


# Comparing similarity of review and product keywords

In [92]:
product_review_similarity_results = []

## BERTScore Comparison

In [94]:
import numpy as np
from bert_score import score
from tqdm import tqdm

def compare_keywords_bert_score(review_keywords, product_keywords):
  """Compares keywords using BERTScore."""

  if not review_keywords or not product_keywords:
    return 0

  max_len = max(len(review_keywords), len(product_keywords))
  review_keywords = (review_keywords * (max_len // len(review_keywords) + 1))[:max_len]
  product_keywords = (product_keywords * (max_len // len(product_keywords) + 1))[:max_len]

  # Calculate BERTScore
  P, R, F1 = score(
    review_keywords,
    product_keywords,
    lang="en",
    model_type="bert-base-uncased",
    verbose=False,
    nthreads=4,
    device="mps",
    use_fast_tokenizer=True
  )

  return F1.mean().item()

tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_bert_score(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'bert_score': similarity_score
        })

        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['bert_score'] for r in product_review_similarity_results if 'bert_score' in r]))

  0%|          | 14/3168 [00:08<26:35,  1.98it/s] 

KeyboardInterrupt: 

## Improved BERTScore Comparison

In [95]:
# prompt: propose other similarity score based on categories, brands and review_keywords; other measure then above, produce sth different ; combine also comaprison between full title of product name and concatenated string from category, brand and keywords from review, print every subsequent result, copare strings with bert score not tiff

def compare_product_review_similarity(review_data, product_data):
  """
  Calculates a similarity score between a review and a product based on
  categories, brands, and keywords, including a comparison of full product
  title with review information using BERTScore.
  """

  similarity_score = 0

  # Category Matching (Highest weight)
  if review_data.get('product category') == product_data.get('product category'):
    similarity_score += 0.5

  # Brand Matching (Medium weight)
  if review_data.get('brand') == product_data.get('brand'):
    similarity_score += 0.3

  # String Comparison (BERTScore) between Product Title and Review Data
  review_info_string = " ".join(
      [
          review_data.get("product category"),
          review_data.get("brand"),
          " ".join(review_data.get("other keywords")),
      ]
  )

  product_info_string = " ".join(
      [
          product_data.get("product category"),
          product_data.get("brand"),
          " ".join(product_data.get("other keywords")),
      ]
  )

  P, R, F1 = score(
      [product_info_string],
      [review_info_string],
      lang="en",
      model_type="bert-base-uncased",
      verbose=False,
      nthreads=4,
      device="mps",
      use_fast_tokenizer=True
  )

  similarity_score += F1.mean().item() * 0.2

  return round(min(1, similarity_score) * 100)


tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))

for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_product_review_similarity(review, product)
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'improved_bert_score': similarity_score
        })
        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['improved_bert_score'] for r in product_review_similarity_results if 'improved_bert_score' in r]))

  0%|          | 1/3168 [00:02<1:55:57,  2.20s/it]

0.691392719745636


  0%|          | 2/3168 [00:02<1:08:28,  1.30s/it]

0.5372622609138489


  0%|          | 3/3168 [00:03<50:38,  1.04it/s]  

...

  0%|          | 6/3168 [00:05<34:40,  1.52it/s]

0.607903003692627


  0%|          | 7/3168 [00:05<34:42,  1.52it/s]

0.5260867476463318


KeyboardInterrupt: 

# Added code

## Cosine Similarity Comparison

In [97]:
import numpy as np
from sentence_transformers import SentenceTransformer

def compare_keywords_cosine_similarity(review_keywords, product_keywords):
    if not review_keywords or not product_keywords:
        return 0.0

    model = SentenceTransformer('all-MiniLM-L6-v2')  # Choose an appropriate model

    # Compute embeddings
    review_embeddings = model.encode(review_keywords, device="mps")
    product_embeddings = model.encode(product_keywords, device="mps")
    
    # Average the embeddings
    avg_review_embedding = np.mean(review_embeddings, axis=0)
    avg_product_embedding = np.mean(product_embeddings, axis=0)

    # Compute cosine similarity
    similarity = np.dot(avg_review_embedding, avg_product_embedding) / (
        np.linalg.norm(avg_review_embedding) * np.linalg.norm(avg_product_embedding)
    )

    return similarity

tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_cosine_similarity(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'cosine_similarity': similarity_score
        })

        bar.update(1)
    
bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['cosine_similarity'] for r in product_review_similarity_results if 'cosine_similarity' in r]))

  1%|          | 17/3168 [00:27<1:19:52,  1.52s/it]

KeyboardInterrupt: 

## Jaccard Similarity Comparison

In [105]:
def compare_keywords_jaccard_similarity(review_keywords, product_keywords):
    set_review = set(review_keywords)
    set_product = set(product_keywords)
    intersection = set_review.intersection(set_product)
    union = set_review.union(set_product)
    if not union:
        return 0.0
    return len(intersection) / len(union)

tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_jaccard_similarity(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'jaccard_similarity': similarity_score
        })

        bar.update(1)
        
bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['jaccard_similarity'] for r in product_review_similarity_results if 'jaccard_similarity' in r]))

100%|██████████| 3168/3168 [00:00<00:00, 629562.92it/s]

Mean similarity between review and product keywords: 0.010297427297152857


## Levenshtein Distance Comparison

In [106]:
import Levenshtein

def compare_keywords_levenshtein_distance(review_keywords, product_keywords):
    total_distance = 0
    comparisons = 0
    for r_keyword in review_keywords:
        for p_keyword in product_keywords:
            distance = Levenshtein.distance(r_keyword, p_keyword)
            total_distance += distance
            comparisons += 1
    if comparisons == 0:
        return 0.0
    # Normalize the distance
    average_distance = total_distance / comparisons
    max_length = max(len(''.join(review_keywords)), len(''.join(product_keywords)))
    similarity = 1 - (average_distance / max_length)
    return similarity

tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_levenshtein_distance(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'levenshtein_distance': similarity_score
        })

        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['levenshtein_distance'] for r in product_review_similarity_results if 'levenshtein_distance' in r]))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.9 MB/s eta 0:00:00a 0:00:01


100%|██████████| 3168/3168 [00:00<00:00, 42873.32it/s]

Mean similarity between review and product keywords: 0.8723675850178576


## Overlap Coefficient Comparison

In [107]:
def compare_keywords_overlap_coefficient(review_keywords, product_keywords):
    set_review = set(review_keywords)
    set_product = set(product_keywords)
    intersection = set_review.intersection(set_product)
    min_size = min(len(set_review), len(set_product))
    if min_size == 0:
        return 0.0
    return len(intersection) / min_size


tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_overlap_coefficient(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'overlap_coefficient': similarity_score
        })

        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['overlap_coefficient'] for r in product_review_similarity_results if 'overlap_coefficient' in r]))

100%|██████████| 3168/3168 [00:00<00:00, 520549.83it/s]

Mean similarity between review and product keywords: 0.023836602861224073


## BLEU Score Comparison

In [108]:
from nltk.translate.bleu_score import sentence_bleu

def compare_keywords_bleu_score(review_keywords, product_keywords):
    if not review_keywords or not product_keywords:
        return 0.0
    score = sentence_bleu([product_keywords], review_keywords)
    return score

tqdm._instances.clear() 
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_bleu_score(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'bleu_score': similarity_score
        })

        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['bleu_score'] for r in product_review_similarity_results if 'bleu_score' in r]))

  0%|          | 0/3168 [00:00<?, ?it/s]/opt/anaconda3/envs/workspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/anaconda3/envs/workspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/anaconda3/envs/workspace/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower 

Mean similarity between review and product keywords: 3.368273056482451e-81


## ROUGE Score Comparison

In [109]:
from rouge import Rouge

def compare_keywords_rouge_score(review_keywords, product_keywords):
    if not review_keywords or not product_keywords:
        return 0.0
    rouge = Rouge()
    scores = rouge.get_scores(' '.join(review_keywords), ' '.join(product_keywords))
    return scores[0]['rouge-l']['f']


tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))
for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_rouge_score(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'rouge_score': similarity_score
        })

        bar.update(1)

bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['rouge_score'] for r in product_review_similarity_results if 'rouge_score' in r]))

100%|██████████| 3168/3168 [00:00<00:00, 12267.27it/s]

Mean similarity between review and product keywords: 0.03231193136848701


## TF-IDF Cosine Similarity Comparison

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compare_keywords_tfidf_cosine_similarity(review_keywords, product_keywords):
    corpus = [' '.join(review_keywords), ' '.join(product_keywords)]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

tqdm._instances.clear()
bar = tqdm(total=len(merged_reviews) * len(merged_products))

for review in merged_reviews:
    for product in merged_products:
        similarity_score = compare_keywords_tfidf_cosine_similarity(review['other keywords'], product['other keywords'])
        product_review_similarity_results.append({
            'review': review['review'],
            'product': product['product'],
            'tfidf_cosine_similarity': similarity_score
        })

        bar.update(1)
        
bar.close()
print("Mean similarity between review and product keywords:", np.mean([r['tfidf_cosine_similarity'] for r in product_review_similarity_results if 'tfidf_cosine_similarity' in r]))

100%|██████████| 3168/3168 [00:02<00:00, 1536.32it/s]

Mean similarity between review and product keywords: 0.028697277894282384


# Merging results from different similarity measures

In [112]:
import pandas as pd

product_review_similarity_df = pd.DataFrame(product_review_similarity_results)
product_review_similarity_df.to_csv('js_zl_nk_js_product_review_similarity.csv', index=False)
product_review_similarity_df

,review,product,bert_score,improved_bert_score,cosine_similarity,jaccard_similarity,levenshtein_distance,overlap_coefficient,bleu_score,rouge_score,tfidf_cosine_similarity
0,Keeps my coffee hot for hours—just what I need...,Contigo Workday Travel Mug – Keeps Coffee Hot ...,0.507972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Keeps my coffee hot for hours—just what I need...,Zojirushi Sleek Travel Mug – Perfect Fit for C...,0.500482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Keeps my coffee hot for hours—just what I need...,Hydro Flask Lightweight Insulation Mug – Stay ...,0.545319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Keeps my coffee hot for hours—just what I need...,Yeti Thermal Mug – No More Cold Coffee!,0.503856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Keeps my coffee hot for hours—just what I need...,Contigo All-Day Heat Retention Mug – Ideal for...,0.435202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
38079,This is my new favorite detergent. So much bet...,Seventh Generation Powder Detergent – Great fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087944
38080,This is my new favorite detergent. So much bet...,Ecover Eco-Friendly Detergent – Perfect for th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103528
38081,This is my new favorite detergent. So much bet...,Gain Powder Detergent – Fresh-Smelling Laundry...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091938
38082,This is my new favorite detergent. So much bet...,"Arm & Hammer Powder Detergent – Great Value, L...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.262097
